The historical data of users is currently stored in an AWS database in the cloud.
Lets create methods in DataExtractor and DatabaseConnector class that will help extract the information from an AWS RDS database.

Step 1:
Create a db_creds.yaml file containing the database credentials, they are as follows:
RDS_HOST: data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com
RDS_PASSWORD: AiCore2022
RDS_USER: aicore_admin
RDS_DATABASE: postgres
RDS_PORT: 5432

Step 2:
Create a method read_db_creds(): this will read the credentials yaml file and return a dictionary of the credentials.
pip install PyYAML and import yaml to do this.

In [1]:
import yaml

In [2]:
def read_db_creds(creds_filename):
    file = open(creds_filename, 'r')
    creds = yaml.safe_load(file)
    return creds

Step 3:
def init_db_engine(): 
- reads the credentials from the return of read_db_creds 
- initialise and return an sqlalchemy database engine.

In [3]:
from sqlalchemy import create_engine

In [4]:
def init_db_engine(creds_filename):
    creds = read_db_creds(creds_filename)    
    HOST = creds['RDS_HOST']
    PORT = creds['RDS_PORT']
    USER = creds['RDS_USER']
    PASSWORD = creds['RDS_PASSWORD']
    DATABASE_TYPE = 'postgresql'
    DBAPI = 'psycopg2'
    DATABASE = creds['RDS_DATABASE']
    engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
    return engine

Step 4: In DataExtractor class:
df list_db_tables(): 
Use the engine from init_db_engine to list all the tables in the database

In [5]:
from sqlalchemy import inspect

In [6]:
def list_db_tables(filename): 
    engine  = init_db_engine(filename)
    # list all the tables
    table_name_ls = inspect(engine).get_table_names()
    return table_name_ls

In [7]:
table_name_ls = list_db_tables('db_creds.yaml')

In [8]:
for i in range (len(table_name_ls)): 
    print (table_name_ls[i])


legacy_store_details
legacy_users
orders_table


Step 5: 
in class DataExtractor: 
def read_rds_table(DatabaseConnector instance, table name):
 extract the database table to a pandas DataFrame.
 return the pd df

It will take in an instance of your DatabaseConnector class and the table name as an argument and return a pandas DataFrame.


In [9]:
import pandas as pd
from sqlalchemy import connectors

In [10]:
def read_rds_table(filename, table_name):
    #engine = init_db_engine(creds_filename)
    #conn = engine.connect()
    #df_data = conn.execute()
    engine = init_db_engine(filename)
    users = pd.read_sql_query(f"select * from {table_name}", engine)
    return users    

In [11]:
users = read_rds_table(filename='db_creds.yaml', table_name='legacy_users')

In [12]:
users.head()

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


Use your list_db_tables method to get the name of the table containing user data.
Use the read_rds_table method to extract the table containing user data and return a pandas DataFrame.

Step 6:
in DataCleaning class: 
def clean_user_data() -> perform the cleaning of the user data.

You will need clean the user data, look out for NULL values, errors with dates, incorrectly typed values and rows filled with the wrong information.

In [13]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [14]:
from datetime import datetime

In [15]:
usercopy = users.copy()

In [16]:
usercopy['date_of_birth']

0        1990-09-30
1        1940-12-01
2        1995-08-02
3        1972-09-23
4        1952-12-20
            ...    
15315    1943-08-09
15316    1948-08-20
15317    1940-10-09
15318    1952-06-04
15319    1994-03-27
Name: date_of_birth, Length: 15320, dtype: object

In [17]:
import numpy as np

##### converting the date of birth column from object type to datetime64[ns]

In [18]:
is_date = pd.to_datetime(usercopy['date_of_birth'], format='%Y-%m-%d', errors='coerce')
usercopy = usercopy[is_date.notna()]

In [19]:
usercopy['date_of_birth'] = usercopy['date_of_birth'].astype('datetime64[ns]')

In [20]:
usercopy['date_of_birth']

0       1990-09-30
1       1940-12-01
2       1995-08-02
3       1972-09-23
4       1952-12-20
           ...    
15315   1943-08-09
15316   1948-08-20
15317   1940-10-09
15318   1952-06-04
15319   1994-03-27
Name: date_of_birth, Length: 15268, dtype: datetime64[ns]

##### converting join_date to datetime64[ns]

In [21]:
usercopy['join_date']

0        2018-10-10
1        2001-12-20
2        2016-12-16
3        2004-02-23
4        2006-09-01
            ...    
15315    2016-04-15
15316    2020-07-20
15317    2021-03-07
15318    2011-01-03
15319    2015-08-28
Name: join_date, Length: 15268, dtype: object

In [22]:
# finding the rows where data is not in correct format
is_dt_form = pd.to_datetime(usercopy['join_date'], format='%Y-%m-%d', errors = 'coerce')
# creating a new df with the is_dt_form series, where it is not NaN 
usercopy = usercopy[is_dt_form.notna()]

In [23]:
# converting the 'join_date' in the new df to datetime
usercopy['join_date'] = usercopy['join_date'].astype('datetime64[ns]')

In [24]:
usercopy['join_date']

0       2018-10-10
1       2001-12-20
2       2016-12-16
3       2004-02-23
4       2006-09-01
           ...    
15315   2016-04-15
15316   2020-07-20
15317   2021-03-07
15318   2011-01-03
15319   2015-08-28
Name: join_date, Length: 15254, dtype: datetime64[ns]

In [25]:
usercopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15254 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          15254 non-null  int64         
 1   first_name     15254 non-null  object        
 2   last_name      15254 non-null  object        
 3   date_of_birth  15254 non-null  datetime64[ns]
 4   company        15254 non-null  object        
 5   email_address  15254 non-null  object        
 6   address        15254 non-null  object        
 7   country        15254 non-null  object        
 8   country_code   15254 non-null  object        
 9   phone_number   15254 non-null  object        
 10  join_date      15254 non-null  datetime64[ns]
 11  user_uuid      15254 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 1.5+ MB


##### replacing wrong values of country code with correct one

In [26]:
usercopy['country_code'] = usercopy['country_code'].astype('category')

In [27]:
usercopy['country_code']

0        DE
1        GB
2        GB
3        GB
4        GB
         ..
15315    GB
15316    GB
15317    DE
15318    US
15319    GB
Name: country_code, Length: 15254, dtype: category
Categories (4, object): ['DE', 'GB', 'GGB', 'US']

In [28]:
usercopy['country_code'].value_counts()

GB     9348
DE     4698
US     1202
GGB       6
Name: country_code, dtype: int64

In [29]:
usercopy['country_code'] = usercopy['country_code'].replace('GGB', 'GB')

In [30]:
usercopy['country_code'].value_counts()

GB    9354
DE    4698
US    1202
Name: country_code, dtype: int64

In [31]:
usercopy['country_code']

0        DE
1        GB
2        GB
3        GB
4        GB
         ..
15315    GB
15316    GB
15317    DE
15318    US
15319    GB
Name: country_code, Length: 15254, dtype: category
Categories (3, object): ['DE', 'GB', 'US']

In [32]:
usercopy['country'] = usercopy['country'].astype('category')

In [33]:
usercopy['country']

0               Germany
1        United Kingdom
2        United Kingdom
3        United Kingdom
4        United Kingdom
              ...      
15315    United Kingdom
15316    United Kingdom
15317           Germany
15318     United States
15319    United Kingdom
Name: country, Length: 15254, dtype: category
Categories (3, object): ['Germany', 'United Kingdom', 'United States']

In [34]:
usercopy['country'].describe()

count              15254
unique                 3
top       United Kingdom
freq                9354
Name: country, dtype: object

### transforming to integers 'country_code', 'phone_number'

In [35]:
usercopy[['phone_number', 'country_code']].head(100)

,phone_number,country_code
0,+49(0) 047905356,DE
1,(0161) 496 0674,GB
2,+44(0)121 4960340,GB
3,(0306) 999 0871,GB
4,0121 496 0225,GB
...,...,...
95,(01756) 82716,DE
96,(00632) 193546,DE
97,+44(0)1614960324,GB
98,+44(0)1174960171,GB


In [36]:
usercopy['phone_number'] = usercopy['phone_number'].str.replace(' ', '')

In [37]:
usercopy['phone_number']

0         +49(0)047905356
1           (0161)4960674
2        +44(0)1214960340
3           (0306)9990871
4             01214960225
               ...       
15315     +44(0)292018946
15316    +44(0)1144960977
15317          0298408192
15318        239.711.3836
15319    +44(0)1314960870
Name: phone_number, Length: 15254, dtype: object

In [38]:
usercopy['phone_number'] = usercopy['phone_number'].str.replace('.', '')

C:\Users\joyet\AppData\Local\Temp\ipykernel_7928\1438632267.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  usercopy['phone_number'] = usercopy['phone_number'].str.replace('.', '')


In [39]:
usercopy['phone_number']

0         +49(0)047905356
1           (0161)4960674
2        +44(0)1214960340
3           (0306)9990871
4             01214960225
               ...       
15315     +44(0)292018946
15316    +44(0)1144960977
15317          0298408192
15318          2397113836
15319    +44(0)1314960870
Name: phone_number, Length: 15254, dtype: object

In [40]:
usercopy['phone_number'].head(10)

0      +49(0)047905356
1        (0161)4960674
2     +44(0)1214960340
3        (0306)9990871
4          01214960225
5    277-664-6389x8405
6           0289018749
7     +44(0)1414960221
8          02890180338
9           6554215915
Name: phone_number, dtype: object

In [41]:
usercopy['phone_number'] = usercopy['phone_number'].str.replace('x', ' ext. ')

In [42]:
usercopy.columns

Index(['index', 'first_name', 'last_name', 'date_of_birth', 'company',
       'email_address', 'address', 'country', 'country_code', 'phone_number',
       'join_date', 'user_uuid'],
      dtype='object')

In [43]:
usercopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15254 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          15254 non-null  int64         
 1   first_name     15254 non-null  object        
 2   last_name      15254 non-null  object        
 3   date_of_birth  15254 non-null  datetime64[ns]
 4   company        15254 non-null  object        
 5   email_address  15254 non-null  object        
 6   address        15254 non-null  object        
 7   country        15254 non-null  category      
 8   country_code   15254 non-null  category      
 9   phone_number   15254 non-null  object        
 10  join_date      15254 non-null  datetime64[ns]
 11  user_uuid      15254 non-null  object        
dtypes: category(2), datetime64[ns](2), int64(1), object(7)
memory usage: 1.3+ MB


In [44]:
def is_uk(x): 
    if x == 'GB':
        return True
    elif x == 'US':
        return False
    else:
        return False

usercopy['uk_number'] = usercopy['country_code'].apply(is_uk)

In [45]:
def is_us(x): 
    if x == 'US':
        return True
    elif x == 'GB':
        return False
    else:
        return False

usercopy['us_number'] = usercopy['country_code'].apply(is_us)

In [46]:
def is_de(x): 
    if x == 'DE':
        return True
    elif x == 'US':
        return False
    else:
        return False

usercopy['de_number'] = usercopy['country_code'].apply(is_de)

In [47]:
usercopy[['country_code', 'country', 'phone_number', 'uk_number', 'us_number', 'de_number']].head(20)

,country_code,country,phone_number,uk_number,us_number,de_number
0,DE,Germany,+49(0)047905356,False,False,True
1,GB,United Kingdom,(0161)4960674,True,False,False
2,GB,United Kingdom,+44(0)1214960340,True,False,False
3,GB,United Kingdom,(0306)9990871,True,False,False
4,GB,United Kingdom,01214960225,True,False,False
5,US,United States,277-664-6389 ext. 8405,False,True,False
6,GB,United Kingdom,0289018749,True,False,False
7,GB,United Kingdom,+44(0)1414960221,True,False,False
8,GB,United Kingdom,02890180338,True,False,False
9,US,United States,6554215915,False,True,False


In [48]:
usercopy['phone_number'] = usercopy['phone_number'].astype('string')

In [49]:
usercopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15254 entries, 0 to 15319
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          15254 non-null  int64         
 1   first_name     15254 non-null  object        
 2   last_name      15254 non-null  object        
 3   date_of_birth  15254 non-null  datetime64[ns]
 4   company        15254 non-null  object        
 5   email_address  15254 non-null  object        
 6   address        15254 non-null  object        
 7   country        15254 non-null  category      
 8   country_code   15254 non-null  category      
 9   phone_number   15254 non-null  string        
 10  join_date      15254 non-null  datetime64[ns]
 11  user_uuid      15254 non-null  object        
 12  uk_number      15254 non-null  bool          
 13  us_number      15254 non-null  bool          
 14  de_number      15254 non-null  bool          
dtypes: bool(3), categor

In [50]:
import re

In [51]:
usercopy['phone_number'] = usercopy['phone_number'].str.replace('\(', '')

C:\Users\joyet\AppData\Local\Temp\ipykernel_7928\274966882.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  usercopy['phone_number'] = usercopy['phone_number'].str.replace('\(', '')


In [52]:
usercopy['phone_number'] = usercopy['phone_number'].str.replace('\)', '')

C:\Users\joyet\AppData\Local\Temp\ipykernel_7928\736711661.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  usercopy['phone_number'] = usercopy['phone_number'].str.replace('\)', '')


In [53]:
usercopy['phone_number'].head(20)

0              +490047905356
1                01614960674
2             +4401214960340
3                03069990871
4                01214960225
5     277-664-6389 ext. 8405
6                 0289018749
7             +4401414960221
8                02890180338
9                 6554215915
10             +441414960404
11               01144960775
12               02890180333
13             +490977574337
14            +4900406372221
15             +441174960765
16                5720688397
17               01144960518
18               08346147221
19             +443069990447
Name: phone_number, dtype: string

In [54]:
usercopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15254 entries, 0 to 15319
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          15254 non-null  int64         
 1   first_name     15254 non-null  object        
 2   last_name      15254 non-null  object        
 3   date_of_birth  15254 non-null  datetime64[ns]
 4   company        15254 non-null  object        
 5   email_address  15254 non-null  object        
 6   address        15254 non-null  object        
 7   country        15254 non-null  category      
 8   country_code   15254 non-null  category      
 9   phone_number   15254 non-null  string        
 10  join_date      15254 non-null  datetime64[ns]
 11  user_uuid      15254 non-null  object        
 12  uk_number      15254 non-null  bool          
 13  us_number      15254 non-null  bool          
 14  de_number      15254 non-null  bool          
dtypes: bool(3), categor

In [69]:
duplicated_phone_number = usercopy.duplicated(subset=['user_uuid', 'phone_number', 'address', 'email_address'], keep = False)

In [70]:
duplicated_phone_number.value_counts()

False    15254
dtype: int64

In [71]:
usercopy[duplicated_phone_number]

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid,uk_number,us_number,de_number


In [72]:
is_valid = lambda x : True if '@' in x else False
valid_email = usercopy['email_address'].apply(is_valid)

In [74]:
valid_email.value_counts()

True    15254
Name: email_address, dtype: int64

In [73]:
usercopy['valid_email'] = valid_email

In [78]:
uk_numbers  = usercopy[usercopy['country_code'] == 'GB']

In [79]:
uk_numbers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9354 entries, 1 to 15319
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          9354 non-null   int64         
 1   first_name     9354 non-null   object        
 2   last_name      9354 non-null   object        
 3   date_of_birth  9354 non-null   datetime64[ns]
 4   company        9354 non-null   object        
 5   email_address  9354 non-null   object        
 6   address        9354 non-null   object        
 7   country        9354 non-null   category      
 8   country_code   9354 non-null   category      
 9   phone_number   9354 non-null   string        
 10  join_date      9354 non-null   datetime64[ns]
 11  user_uuid      9354 non-null   object        
 12  uk_number      9354 non-null   bool          
 13  us_number      9354 non-null   bool          
 14  de_number      9354 non-null   bool          
 15  valid_email    9354 

In [57]:
Step 7:

Now create a method in your DatabaseConnector class called upload_to_db. This method will take in a Pandas DataFrame and table name to upload to as an argument.


Step 8:

Once extracted and cleaned use the upload_to_db method to store the data in your Sales_Data database in a table named dim_users.

SyntaxError: invalid syntax (1476942120.py, line 1)